In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd 
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
#Load the data into Pandas dataframe
df = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: int(x))

df = df.iloc[np.random.permutation(len(df))]

X = df['tweet']
Y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)

tfidf = TfidfVectorizer()
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

type(X_train), type(y_train)

In [11]:
df1 = pd.read_csv('../task3_data/task3_training.tsv', sep='\t')[['type', 'tweet']]
df1 = df1.fillna(0)
df1 = df1.replace('ADR', 1)
df1 = df1.iloc[np.random.permutation(len(df1))]
Train = df1

df2 = pd.read_csv('../task3_data/task3_validation.tsv', sep='\t')[['type', 'tweet']]
df2 = df2.fillna(0)
df2 = df2.replace('ADR', 1)
df2 = df2.iloc[np.random.permutation(len(df2))]
df2 = df2.dropna()
Test = df2

print(Train.shape, Train.shape)
#Train_set = pd.concat([Train_set, Test_set])
print(Test.shape, Test.shape)

X_train = Train['tweet']
y_train = Train['type']

X_test = Test['tweet']
y_test = Test['type']

tfidf = TfidfVectorizer()
tfidf.fit(X_train)

X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)

(2246, 2) (2246, 2)
(560, 2) (560, 2)


In [12]:
from tpot import TPOTClassifier
import joblib
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

tpot = TPOTClassifier(generations=5,
                      population_size=50,
                      verbosity=2,
                      random_state=42,
                      config_dict='TPOT sparse',
                      scoring='f1_macro', use_dask=True, n_jobs=3)
tpot.fit(X_train, y_train)

print()
print('The best TPOT score: ', tpot.score(X_test, y_test))

tpot.export('TPOT_Pipeline.py')

Generation 1 - Current best internal CV score: 0.8107674388811421
Generation 2 - Current best internal CV score: 0.8107674388811421
Generation 3 - Current best internal CV score: 0.8107674388811421
Generation 4 - Current best internal CV score: 0.8107674388811421
Generation 5 - Current best internal CV score: 0.8107674388811421

Best pipeline: BernoulliNB(input_matrix, alpha=0.1, fit_prior=False)

The best TPOT score:  0.75177304964539


In [13]:
with open('TPOT_Pipeline.py') as file:
    for line in file:
        print(line, end='')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=42)

# Average CV score on the training set was: 0.8107674388811421
exported_pipeline = BernoulliNB(alpha=0.1, fit_prior=False)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
